In [1]:
import waterhealer as wh
import json

In [2]:
source = wh.from_kafka(
    ['testing'],
    {
        'bootstrap.servers': 'localhost:9095, localhost:9096',
        'group.id': 'group2',
        'auto.offset.reset': 'latest',
    })

In [3]:
def json_loads(row):
    return (row[0], json.loads(row[1]))

def plus(row):
    no = row[1]['no']
    if no > 12:
        raise Exception('row is bigger than 12')
    return (row[0], no + 1)

In [4]:
outputs = []
source.map(json_loads).map(plus).map(wh.healing, stream = source, callback = print).sink(outputs.append)

Output()

In [5]:
source.start()

In [6]:
from kafka import KafkaProducer
from kafka.partitioner import RoundRobinPartitioner

In [7]:
producer = KafkaProducer(
    bootstrap_servers = ['localhost:9095', 'localhost:9096'],
    api_version = (0, 10),
    partitioner = RoundRobinPartitioner(),
)

In [8]:
producer.bootstrap_connected()

False

In [9]:
from datetime import datetime
import json
from itertools import cycle

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding = 'utf-8')
        value_bytes = bytes(value, encoding = 'utf-8')
        x = producer_instance.send(topic_name, value = value_bytes)
        return True
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))
        return False

## purposely raise the error

Based on function `plus`, if input bigger than 10, it will throw error.

In [10]:
import confluent_kafka as ck

source.consumer.committed([ck.TopicPartition('testing', 0)])[0].offset

-1001

In [11]:
import time

for count in range(14):
    data = {'no': count, 'datetime': str(datetime.now())}
    publish_message(
        producer, 'testing', 'streaming', json.dumps(data)
    )
    print(source.memory, len(source.memory))
    time.sleep(1.0)

ExpiringDict() 0


committed topic: testing partition: 0 offset: 79


1
ExpiringDict() 0
ExpiringDict() 0


committed topic: testing partition: 2 offset: 65


3
ExpiringDict() 0
ExpiringDict() 0


...

ExpiringDict() 0
9
ExpiringDict() 0
ExpiringDict() 0


committed topic: testing partition: 0 offset: 80


11
ExpiringDict() 0
ExpiringDict() 0


committed topic: testing partition: 2 offset: 66


13
ExpiringDict() 0


In [12]:
source.memory

ExpiringDict()

In [14]:
data = {'no': count, 'datetime': str(datetime.now())}
publish_message(
    producer, 'testing', 'streaming', json.dumps(data)
)

True

row is bigger than 12
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/streamz/core.py", line 578, in update
    result = self.func(x, *self.args, **self.kwargs)
  File "<ipython-input-3-f850c9be8189>", line 7, in plus
    raise Exception('row is bigger than 12')
Exception: row is bigger than 12
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x10af19b10>>, <Future finished exception=Exception('row is bigger than 12')>)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/tornado/ioloop.py", line 743, in _run_callback
    ret = callback()
  File "/usr/local/lib/python3.7/site-packages/tornado/ioloop.py", line 767, in _discard_future_result
    future.result()
  File "/usr/local/lib/python3.7/site-packages/tornado/gen.py", line 748, in run
    yielded = self.gen.send(value)
  File "/Users/huseinzolkepli/Documents/wat

In [15]:
source.memory

ExpiringDict([('4-51-testing',
               {'partition': 4, 'offset': 51, 'topic': 'testing'})])

In [16]:
source.consumer.committed([ck.TopicPartition('testing', 4)])[0].offset

51

no update! That is what we want!